In [1]:
# torch-neuronx requires 1.13.1 or more above

# https://github.com/clovaai/donut/issues/132#issuecomment-1412385775
!pip install protobuf==3.19.6 --quiet
!pip install pytorch-lightning==1.6.4 --quiet
!pip install transformers --quiet
!pip install torchvision --quiet
!pip install torch==1.13.1 --quiet
!pip install timm==0.5.4 --quiet
!pip install donut-python==1.0.9 --quiet
!pip install neuronx-cc==2.* torch-neuronx --quiet

In [2]:
!dpkg -l aws-neuronx-runtime-lib

Desired=Unknown/Install/Remove/Purge/Hold
| Status=Not/Inst/Conf-files/Unpacked/halF-conf/Half-inst/trig-aWait/Trig-pend
|/ Err?=(none)/Reinst-required (Status,Err: uppercase=bad)
||/ Name                    Version            Architecture Description
+++-=======================-==================-============-=================================
ii  aws-neuronx-runtime-lib 2.13.6.0-29de104d6 amd64        neuron_runtime built using CMake


In [3]:
!pip list |grep -e torch -e neuron -e donut

aws-neuronx-runtime-discovery 2.9
donut-python                  1.0.9
libneuronxla                  0.5.207
neuronx-cc                    2.6.0.19+3d819e565
neuronx-hwm                   2.6.0.0+826e77395
pytorch-lightning             1.6.4
torch                         1.13.1
torch-neuronx                 1.13.1.1.7.0
torch-xla                     1.13.1+torchneuron6
torchmetrics                  0.11.4
torchvision                   0.14.1


In [4]:
import datetime
import subprocess

# 現在の日時を取得
now = datetime.datetime.now()
date_str = now.strftime('%Y%m%d_%H%M%S')

# Pythonバージョンの確認
python_version = subprocess.run(['python', '--version'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True).stdout.strip()

# CUDAバージョンの確認
try:
    nvcc_version = subprocess.run(['nvcc', '--version'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True).stdout.strip()
    _, cuda_version = nvcc_version.split('\n')[-2].split(',')[1].split()
except:
    cuda_version = 'CPU'

# torchバージョンの確認
import torch
torch_version = torch.__version__

# transformersバージョンの確認
import transformers
transformers_version = transformers.__version__

# 実行時のコマンドと実行結果を保存するファイル名
log_file = f"diffusion_{date_str}.log"


print(f"Python version: {python_version}\n")
print(f"CUDA version: {cuda_version}\n")
print(f"torch version: {torch_version}\n")
print(f"transformers version: {transformers_version}\n\n")

/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Python version: Python 3.8.10

CUDA version: 11.8

torch version: 1.13.1+cu117

transformers version: 4.11.3




In [5]:
import torch
from PIL import Image
import sys
from donut import DonutModel

import IPython
import pprint
from donut.model import DonutConfig, DonutModel, SwinEncoder, BARTDecoder

In [6]:
import math
import os
import re
from typing import Any, List, Optional, Union

import numpy as np
import PIL
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import ImageOps
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.models.swin_transformer import SwinTransformer
from torchvision import transforms
from torchvision.transforms.functional import resize, rotate
from transformers import MBartConfig, MBartForCausalLM, XLMRobertaTokenizer
from transformers.file_utils import ModelOutput
from transformers.modeling_utils import PretrainedConfig, PreTrainedModel

In [7]:
import torch
import torch_neuronx
import os
import numpy as np
from torch.nn import functional as F

2023-May-30 09:26:48.0600 498175:498175 ERROR  TDRV:tdrv_get_dev_info                       No neuron device available


In [8]:
import torch
from PIL import Image
import sys

import IPython
import pprint

In [9]:
from donut import DonutModel

In [10]:
!rm -rf test_01.jpg

# 英語のスライド
!wget -c https://media.slidesgo.com/storage/162635/conversions/1-market-share-infographics-thumb.jpg \
      -O ./test_01.jpg

# 日本語のレシート
# !wget -c https://www.isp21.co.jp/wp-content/uploads/solution/library/library02-1.jpg \
#       -O ./test_01.jpg

input_img_path = './test_01.jpg'

--2023-05-30 09:32:39--  https://media.slidesgo.com/storage/162635/conversions/1-market-share-infographics-thumb.jpg
Resolving media.slidesgo.com (media.slidesgo.com)... 104.22.1.146, 172.67.9.18, 104.22.0.146, ...
Connecting to media.slidesgo.com (media.slidesgo.com)|104.22.1.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43098 (42K) [image/jpeg]
Saving to: ‘./test_01.jpg’

./test_01.jpg       100%[===================>]  42.09K  --.-KB/s    in 0.001s  

2023-05-30 09:32:39 (36.2 MB/s) - ‘./test_01.jpg’ saved [43098/43098]



In [11]:
donut_model = DonutModel.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip").to("cpu")
donut_model.eval() 

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████| 401/401 [00:00<00:00, 2.47MB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████████████████| 965M/965M [00:10<00:00, 98.5MB/s]
/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████| 1.24M/1.24M [00:00<00:00, 87.6MB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████| 536/536 [00:00<00:00, 3.61MB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████| 3.83M/3.83M [00:00<00:00, 98.

DonutModel(
  (encoder): SwinEncoder(
    (model): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (0): BasicLayer(
          dim=128, input_resolution=(640, 480), depth=2
          (blocks): ModuleList(
            (0): SwinTransformerBlock(
              (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attn): WindowAttention(
                (qkv): Linear(in_features=128, out_features=384, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=128, out_features=128, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop_path): Identity()
              (norm2): LayerNorm((128,), ep

In [12]:
task_prompt = "<s_rvlcdip>"

input_img = Image.open("./test_01.jpg")
output = donut_model.inference(image=input_img, prompt=task_prompt)["predictions"][0]

In [13]:
output

{'class': 'presentation'}

In [14]:
from timm.models.swin_transformer import WindowAttention, SwinTransformer

In [16]:
window_attention = WindowAttention(dim=1024, num_heads=32, window_size=(10, 10), qkv_bias=True, attn_drop=0.0, proj_drop=0.0) 

In [17]:
dummy_input = torch.zeros([48, 100, 1024])

In [18]:
window_attention.forward(dummy_input) 

tensor([[[-0.0068, -0.0451,  0.0154,  ..., -0.0066,  0.0085,  0.0078],
         [-0.0068, -0.0451,  0.0154,  ..., -0.0066,  0.0085,  0.0078],
         [-0.0068, -0.0451,  0.0154,  ..., -0.0066,  0.0085,  0.0078],
         ...,
         [-0.0068, -0.0451,  0.0154,  ..., -0.0066,  0.0085,  0.0078],
         [-0.0068, -0.0451,  0.0154,  ..., -0.0066,  0.0085,  0.0078],
         [-0.0068, -0.0451,  0.0154,  ..., -0.0066,  0.0085,  0.0078]],

        [[-0.0068, -0.0451,  0.0154,  ..., -0.0066,  0.0085,  0.0078],
         [-0.0068, -0.0451,  0.0154,  ..., -0.0066,  0.0085,  0.0078],
         [-0.0068, -0.0451,  0.0154,  ..., -0.0066,  0.0085,  0.0078],
         ...,
         [-0.0068, -0.0451,  0.0154,  ..., -0.0066,  0.0085,  0.0078],
         [-0.0068, -0.0451,  0.0154,  ..., -0.0066,  0.0085,  0.0078],
         [-0.0068, -0.0451,  0.0154,  ..., -0.0066,  0.0085,  0.0078]],

        [[-0.0068, -0.0451,  0.0154,  ..., -0.0066,  0.0085,  0.0078],
         [-0.0068, -0.0451,  0.0154,  ..., -0

In [19]:
torch.jit.trace(window_attention, dummy_input) 

WindowAttention(
  original_name=WindowAttention
  (qkv): Linear(original_name=Linear)
  (attn_drop): Dropout(original_name=Dropout)
  (proj): Linear(original_name=Linear)
  (proj_drop): Dropout(original_name=Dropout)
  (softmax): Softmax(original_name=Softmax)
)

In [20]:
torch_neuronx.trace(window_attention, dummy_input) 

NeuronModule(original_name=NeuronModule)

In [21]:
from timm.models.swin_transformer import SwinTransformerBlock

In [23]:
stb = SwinTransformerBlock(dim=1024, input_resolution=(80, 60), num_heads=32, window_size=10, shift_size=5, mlp_ratio=4.0, qkv_bias=True, drop=0.0, attn_drop=0.0) 

In [24]:
dummy_input = torch.zeros((1, 4800, 1024))

In [25]:
stb(dummy_input) 

tensor([[[-0.2110,  0.1001,  0.2346,  ...,  0.1103, -0.0052, -0.1115],
         [-0.2110,  0.1001,  0.2346,  ...,  0.1103, -0.0052, -0.1115],
         [-0.2110,  0.1001,  0.2346,  ...,  0.1103, -0.0052, -0.1115],
         ...,
         [-0.2110,  0.1001,  0.2346,  ...,  0.1103, -0.0052, -0.1115],
         [-0.2110,  0.1001,  0.2346,  ...,  0.1103, -0.0052, -0.1115],
         [-0.2110,  0.1001,  0.2346,  ...,  0.1103, -0.0052, -0.1115]]],
       grad_fn=<AddBackward0>)

In [26]:
torch.jit.trace(stb, dummy_input) 

/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch/__init__.py:853: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message
/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/timm/models/swin_transformer.py:119: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  B = int(windows.shape[0] / (H * W / window_size / window_size))


SwinTransformerBlock(
  original_name=SwinTransformerBlock
  (norm1): LayerNorm(original_name=LayerNorm)
  (attn): WindowAttention(
    original_name=WindowAttention
    (qkv): Linear(original_name=Linear)
    (attn_drop): Dropout(original_name=Dropout)
    (proj): Linear(original_name=Linear)
    (proj_drop): Dropout(original_name=Dropout)
    (softmax): Softmax(original_name=Softmax)
  )
  (drop_path): Identity(original_name=Identity)
  (norm2): LayerNorm(original_name=LayerNorm)
  (mlp): Mlp(
    original_name=Mlp
    (fc1): Linear(original_name=Linear)
    (act): GELU(original_name=GELU)
    (drop1): Dropout(original_name=Dropout)
    (fc2): Linear(original_name=Linear)
    (drop2): Dropout(original_name=Dropout)
  )
)

In [27]:
torch_neuronx.trace(stb, dummy_input)

NeuronModule(original_name=NeuronModule)

In [28]:
swin_transformer = SwinTransformer() 

In [29]:
dummy_input = torch.zeros((1, 3, 224, 224))

In [30]:
swin_transformer(dummy_input) 

tensor([[ 1.0582e-01,  1.3340e-01, -1.2882e-01,  1.0457e+00, -2.0120e-01,
          1.5382e-01, -6.9004e-01, -4.4204e-01, -2.1146e-02,  7.9767e-01,
         -6.1969e-01, -3.8736e-01,  4.5748e-01,  2.0448e-01,  1.1823e+00,
         -5.0816e-01,  1.1756e+00,  1.3605e-01, -2.7575e-01, -1.1183e-01,
          6.5692e-01,  9.7905e-01, -4.2008e-01,  5.1391e-02, -3.2915e-01,
          3.0060e-01, -1.1677e-01, -5.9148e-01,  4.1320e-01,  7.2589e-01,
         -8.1697e-01,  5.4945e-02,  2.5633e-01,  8.1152e-01,  7.1605e-01,
          8.2939e-01, -5.6233e-02,  2.9957e-01,  8.5922e-01, -7.0553e-01,
          7.7346e-01,  4.3777e-01,  3.2396e-01, -9.0288e-02, -1.2218e+00,
          6.8019e-01, -6.8559e-02,  4.8461e-02, -4.0518e-01,  3.7049e-01,
          4.4140e-01, -1.6573e-01,  5.7697e-02,  6.8767e-01,  1.9585e-01,
         -4.9692e-01, -9.7986e-02,  2.8836e-02, -8.2742e-01, -9.5816e-02,
         -7.4876e-01, -7.7008e-01,  3.1343e-01, -7.6634e-01, -8.0548e-01,
         -1.2685e-01,  3.1898e-01, -4.

In [31]:
torch.jit.trace(swin_transformer, dummy_input) 

/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch/jit/_trace.py:1001: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1000 / 1000 (100.0%)
Greatest absolute difference: 0.9971470087766647 at index (0, 5) (up to 1e-05 allowed)
Greatest relative difference: 2493.6227840690804 at index (0, 296) (up to 1e-05 allowed)
  _check_trace(


SwinTransformer(
  original_name=SwinTransformer
  (patch_embed): PatchEmbed(
    original_name=PatchEmbed
    (proj): Conv2d(original_name=Conv2d)
    (norm): LayerNorm(original_name=LayerNorm)
  )
  (pos_drop): Dropout(original_name=Dropout)
  (layers): Sequential(
    original_name=Sequential
    (0): BasicLayer(
      original_name=BasicLayer
      (blocks): ModuleList(
        original_name=ModuleList
        (0): SwinTransformerBlock(
          original_name=SwinTransformerBlock
          (norm1): LayerNorm(original_name=LayerNorm)
          (attn): WindowAttention(
            original_name=WindowAttention
            (qkv): Linear(original_name=Linear)
            (attn_drop): Dropout(original_name=Dropout)
            (proj): Linear(original_name=Linear)
            (proj_drop): Dropout(original_name=Dropout)
            (softmax): Softmax(original_name=Softmax)
          )
          (drop_path): Identity(original_name=Identity)
          (norm2): LayerNorm(original_name=Layer

In [32]:
swin_transformer_neuron = torch_neuronx.trace(swin_transformer, dummy_input) 